# Uchazeč: data o uchazečích o studium na pedagogických fakultách

## Importy

In [1]:
# nejake standardni importy
import os
import sys
import pyreadstat
import pandas as pd
import numpy as np
import re
from statsmodels.stats.weightstats import DescrStatsW
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
# aby grafy byly rozumně čitelné na obrazovce
plt.rcParams['figure.dpi'] = 90
plt.rcParams['axes.titlesize'] = 11
plt.rcParams['figure.figsize'] = 10, 5
#plt.ioff()
import dbf

In [141]:
data_root = '/mnt/d/projects/idea/data'
path17 = 'uchazec/0022MUCH17P'
path21 = 'uchazec/0022MUCH21P'

In [142]:
# načti data z roku 2017
df = pd.read_csv(f'{data_root}/{path17}.csv', encoding='cp1250')

/tmp/ipykernel_417/2773192003.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path17}.csv', encoding='cp1250')


In [143]:
df.shape

(253584, 28)

In [144]:
df.head(100).to_csv('temp/uchazec/df17_100.csv')

## Loader

In [303]:
def loader(year=17):
    # year can be 17 or 21
    data_root = '/mnt/d/projects/idea/data'
    path = f'uchazec/0022MUCH{year}P'
    
    df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')

    # conversion to numeric
    for c in ['RMAT', 'STAT', 'STATB', 'IZOS'] + [f'APRO{i}' for i in range(1, 4)]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        
    # strip white spaces
    for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
        df[c] = df[c].str.strip()
    
    # label STAT, STATB: register AAST
    aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
    aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')
    aast['ISO_ALPHA3'] = aast['SPC'].str[2:5]
    df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STAT', 'ZKR': 'STAT_LABEL', 'ISO_ALPHA3': 'STAT_ISO'}), 
              how='left')
    df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STATB', 'ZKR': 'STATB_LABEL', 'ISO_ALPHA3': 'STATB_ISO'}), 
              how='left')
    
    # label OBSS - I am not including full school info
    #   -> actually, this give me slightly better match - probably because I am including also specializations that are not valid anymore
    li = []
    for y in range(1999, 2023):
        pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
        li.append(pdf)
    ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)
    
    def selector(frame):
        # just use the longest one - the shorter ones typically use abbreviations
        frame['len'] = frame['PLN_NAZ'].str.strip().str.len()
        return frame.sort_values('len', ascending=False)['PLN_NAZ'].iloc[0]

    ss['OBOR'] = ss['OBOR'].str.strip()
    obory = ss[['OBOR', 'PLN_NAZ']].drop_duplicates()
    obory_uni = obory.groupby('OBOR')[['PLN_NAZ']].apply(selector).reset_index()
    df = pd.merge(df, obory_uni.rename(columns={'OBOR': 'OBSS', 0: 'OBSS_LABEL'}), how='left')
    
    sss = ss[['IZO', 'ZAR_PLN', 'VUSC']].drop_duplicates()
    df = pd.merge(df, sss.rename(columns={'IZO': 'IZOS', 'ZAR_PLN': 'IZOS_LABEL', 'VUSC': 'IZOS_NUTS'}), how='left')
    
    registers = {
        'ODHL': 'http://stistko.uiv.cz/katalog/textdata/C213729MCPP.xml',
        'TYP_ST': 'http://stistko.uiv.cz/katalog/textdata/C214019PASP.xml',
        'FORMA_ST' : 'http://stistko.uiv.cz/katalog/textdata/C214048PAFS.xml',
        'VYPR': 'http://stistko.uiv.cz/katalog/textdata/C214147MCPR.xml',
        'ZAPS': 'http://stistko.uiv.cz/katalog/textdata/C21427MCZR.xml',
        # 'OBSS': 'http://stistko.uiv.cz/katalog/textdata/C113922AKSO.xml'
    }
    
    for c, url in registers.items():
        rg = pd.read_xml(url, encoding='cp1250', xpath='./veta')
        df = pd.merge(df, rg[['KOD', 'TXT']].rename(columns={'KOD': c, 'TXT': f'{c}_LABEL'}), how='left')

    regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
    regpro['RID'] = regpro['RID'].str.strip()
    regpro_cols = ['ZAR_PLN', 'VS_PLN', 'ZAR_NAZ', 'UZEMI']
    regpro = regpro[['RID'] + regpro_cols].rename(columns={x: f'RID_{x}' for x in regpro_cols})
    df = pd.merge(df, regpro.rename(columns={'RID_UZEMI': 'RID_NUTS'}), how='left')
    
    program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
    df = pd.merge(df, program.rename(columns={'KOD': 'PROGRAM', 'NAZEV': 'PROGRAM_LABEL'}), how='left')
    
    # plné obory
    akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
    akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
    for i in range(1, 6):
        df = pd.merge(df, akvo[['KOD', 'TXT']].rename(columns={'KOD': f'OBOR{i}', 'TXT': f'OBOR{i}_LABEL'}), how='left')
        
    # aprobace
    aaap_xml = 'http://stistko.uiv.cz/katalog/textdata/C214234AAAP.xml'
    aaap = pd.read_xml(aaap_xml, encoding='cp1250', xpath='./veta')
    for i in range(1, 4):
        df = pd.merge(df, aaap[['KOD', 'TXT']].rename(columns={'KOD': f'APRO{i}', 'TXT': f'APRO{i}_LABEL'}), how='left')
        
    # studijní obory podle skupin
    akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
    akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')
    for i in [1, 2, 5]:
        df[f'OBOR1_{i}'] = df['OBOR1'].str[:i]
        df = pd.merge(df, akko[['KOD', 'TXT']].rename(columns={'KOD': f'OBOR1_{i}', 'TXT': f'OBOR1_{i}_LABEL'}), how='left')
    
    df['TYP_SS'] = np.where(df['OBSS'] == '', 'JINE', np.where(df['OBSS'].str.startswith('794'), 'G', 'SOS'))
    df['TYP_G'] = np.where(df['TYP_SS'] != 'G', np.nan, df['OBSS'].str[5])
    
    # fillna in some columns
    for c in ['PROGRAM_LABEL'] + [f'OBOR{i}_LABEL' for i in range(1, 6)] + [f'APRO{i}_LABEL' for i in range(1, 4)]:
        df[c] = df[c].fillna('')
    
    return df

In [304]:
df17 = loader()

/tmp/ipykernel_417/678624935.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')


In [305]:
df21 = loader(year=21)

/tmp/ipykernel_417/678624935.py:6: DtypeWarning: Columns (3,6,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')


In [300]:
variable_labels = {
    'RDAT': 'Rozhodné datum sběru',
    'RID': 'Identifikátor fakulty',
    'RID_VS_PLN': 'Název vysoké školy',
    'RID_ZAR_PLN': 'Název fakulty',
    'RID_ZAR_NAZ': 'Zkratka fakulty',
    'RID_NUTS': 'Lokalita fakulty',
    'ROD_KOD': 'Identifikátor uchazeče (kódované rodné číslo)',
    'STATB': 'Stát trvalého pobytu',
    'STATB_LABEL': 'Stát trvalého pobytu',
    'STATB_ISO': 'Stát trvalého pobytu - ISO 3166 alpha-3 kód',
    'OBECB': 'Obec trvalého pobytu',
    'PSCB': 'PSČ trvalého pobytu',
    'STAT': 'Státní příslušnost uchazeče',
    'STAT_LABEL': 'Státní příslušnost uchazeče',
    'STAT_ISO': 'Státní příslušnost uchazeče - ISO 3166 alpha-3 kód',
    'ODHL': 'Odkud se uchazeč hlásí',
    'ODHL_LABEL': 'Odkud se uchazeč hlásí',
    'IZOS': 'Identifikátor střední školy',
    'IZOS_LABEL': 'Název střední školy',
    'IZOS_NUTS': 'Lokalita střední školy',
    'OBSS': 'Obor studia střední školy',
    'OBSS_LABEL': 'Obor studia střední školy',
    'RMAT': 'Rok maturity',
    'TYP_ST': 'Typ studia',
    'TYP_ST_LABEL': 'Typ studia',
    'FORMA_ST': 'Forma studia',
    'FORMA_ST_LABEL': 'Forma studia',
    'DAT_REG': 'Datum podání (registrace) přihlášky',
    'VYPR': 'Výsledek přijímacího řízení',
    'VYPR_LABEL': 'Výsledek přijímacího řízení',
    'DAT_VYPR': 'Datum rozhodnutí o výsledku přijímacího řízení',
    'ZAPS': 'Výsledek zápisu',
    'ZAPS_LABEL': 'Výsledek zápisu',
    'DAT_ZAPS': 'Datum zápisu',
    'PROGRAM': 'Studijní program',
    'OBOR1': 'Studijní obor',
    'OBOR2': 'Studijní obor',
    'OBOR3': 'Studijní obor',
    'OBOR4': 'Studijní obor',
    'OBOR5': 'Studijní obor',
    'APRO1': 'Aprobace',
    'APRO2': 'Aprobace',
    'APRO3': 'Aprobace',
    'PROGRAM_LABEL': 'Studijní program',
    'OBOR1_LABEL': 'Studijní obor',
    'OBOR2_LABEL': 'Studijní obor',
    'OBOR3_LABEL': 'Studijní obor',
    'OBOR4_LABEL': 'Studijní obor',
    'OBOR5_LABEL': 'Studijní obor',
    'APRO1_LABEL': 'Aprobace',
    'APRO2_LABEL': 'Aprobace',
    'APRO3_LABEL': 'Aprobace',
    'OBOR1_1': 'Hlavní studijní obor - skupina',
    'OBOR1_1_LABEL': 'Hlavní studijní obor - skupina',
    'OBOR1_2': 'Hlavní studijní obor - podskupina',
    'OBOR1_2_LABEL': 'Hlavní studijní obor - podskupina',
    'OBOR1_5': 'Hlavní studijní obor - program',
    'OBOR1_5_LABEL': 'Hlavní studijní obor - program',
}

In [307]:
df17[variable_labels.keys()].to_stata(f'{data_root}/uchazec/uch17.dta', write_index=False, version=118, variable_labels=variable_labels)

In [ ]:
df21[variable_labels.keys()].to_stata(f'{data_root}/uchazec/uch21.dta', write_index=False, version=118, variable_labels=variable_labels)

In [308]:
strl_cols = ['RID_VS_PLN', 'RID_ZAR_PLN', 'RID_ZAR_NAZ'] + [x for x in variable_labels if x.endswith('_LABEL')]

In [309]:
df17[variable_labels.keys()].to_stata(f'{data_root}/uchazec/uch17.dta', write_index=False, version=118,
                                      variable_labels=variable_labels, convert_strl=strl_cols)

TypeError: encoding without a string argument

In [287]:
df21[(df21['STAT_ISO'] == 'CZE') & (df21['STATB_ISO'] == 'CZE') & (df21['RMAT'] == 2021)].head(1000).to_csv('temp/uchazec/cz21_.csv')

In [312]:
df21[(df21['STAT_ISO'] == 'CZE') & (df21['STATB_ISO'] == 'CZE')]['ROD_KOD'].str[:4].value_counts()[:30]

0255    23427
0254    23340
0253    23184
0160    21945
0251    21576
0159    21372
0162    20429
0161    20281
0252    20249
0256    19125
0257    17263
0258    14006
0109    13276
0158    13235
0110    12829
0204    12627
0201    12085
0203    12007
0111    11927
0202    11607
0157    11479
0108    11061
0112    10956
0107    10400
0205    10296
0156    10287
0155     9142
0206     8494
0154     8099
0106     7814
Name: ROD_KOD, dtype: int64

In [313]:
df17[(df17['STAT_ISO'] == 'CZE') & (df17['STATB_ISO'] == 'CZE')]['ROD_KOD'].str[:4].value_counts()[:30]

9854    20556
9853    19677
9855    18893
9759    18748
9856    18566
9760    18085
9851    17999
9762    17726
9852    17230
9761    17151
9857    16091
9758    13653
9757    12187
9858    11290
9710    11010
9709    10804
9803    10602
9804     9849
9801     9635
9712     9581
9756     9470
9711     9329
9755     9063
9708     9057
9805     9006
9802     8889
9707     8530
9754     8274
9806     7525
9706     7146
Name: ROD_KOD, dtype: int64

In [260]:
df['TYP_G'].value_counts()

4    63435
8    39691
6    10522
0      809
5      213
7       71
Name: TYP_G, dtype: int64

In [266]:
(~pd.isna(df['OBSS_LABEL'])).value_counts()

True     209764
False     43820
Name: OBSS_LABEL, dtype: int64

In [262]:
(~pd.isna(df['OBSS_LABEL'])).value_counts()

True     211426
False     42158
Name: OBSS_LABEL, dtype: int64

In [279]:
df[(df['STAT_ISO'] == 'CZE') & (df['STATB_ISO'] == 'CZE') & (df['RMAT'] == 2017)].head(1000).to_csv('temp/uchazec/czmat_1000.csv')

In [238]:
df.head()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,RID_ZAR_NAZ,RID_UZEMI,OBOR1_LABEL,OBOR2_LABEL,OBOR3_LABEL,OBOR4_LABEL,OBOR5_LABEL,APRO1_LABEL,APRO2_LABEL,APRO3_LABEL
0,20171031,78900,,840.0,,,840.0,2,999999840,,...,Celoškolská pracoviště,CZ0111,Mezinárodní vztahy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20171031,78900,,368.0,,,368.0,2,999999368,,...,Celoškolská pracoviště,CZ0111,Visual Art Studies,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20171031,15110,000101,112.0,,,112.0,2,999999112,,...,Lékařská fakulta,CZ0712,Všeobecné lékařství,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20171031,15110,000101,112.0,,,112.0,2,999999112,,...,Lékařská fakulta,CZ0712,Zubní lékařství,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20171031,14110,000101,112.0,,23104,112.0,9,999999112,,...,Lékařská fakulta,CZ0622,Všeobecné lékařství,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Číselníky

### Státní příslušnost: STAT, STATB

In [145]:
# státy - číselník AAST - sloupce STATB, STAT
aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')
aast.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AAST,0,Zatím nezjištěno,Zatím nezjištěno,1.1.2000,31.12.2999,***************,pouze pro prezentaci
1,AAST,4,Afghánistán,Afghánská islámská republika,1.7.2014,31.12.2999,AFAFG601 5166020,None
2,AAST,8,Albánie,Albánská republika,5.5.2005,31.12.2999,ALALB111 4107015,None
3,AAST,10,Antarktida,Antarktida,5.5.2005,31.12.2999,AQATA961 5089150,None
4,AAST,12,Alžírsko,Alžírská demokratická a lidová republika,1.1.2011,31.12.2999,DZDZA701 5120830,None


In [146]:
aast['ISO_ALPHA3'] = aast['SPC'].str[2:5]

In [109]:
aast.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN,ISO_ALPHA3
0,AAST,0,Zatím nezjištěno,Zatím nezjištěno,1.1.2000,31.12.2999,***************,pouze pro prezentaci,***
1,AAST,4,Afghánistán,Afghánská islámská republika,1.7.2014,31.12.2999,AFAFG601 5166020,None,AFG
2,AAST,8,Albánie,Albánská republika,5.5.2005,31.12.2999,ALALB111 4107015,None,ALB
3,AAST,10,Antarktida,Antarktida,5.5.2005,31.12.2999,AQATA961 5089150,None,ATA
4,AAST,12,Alžírsko,Alžírská demokratická a lidová republika,1.1.2011,31.12.2999,DZDZA701 5120830,None,DZA


In [147]:
df['STAT'] = pd.to_numeric(df['STAT'], errors='coerce')
df['STATB'] = pd.to_numeric(df['STATB'], errors='coerce')

In [148]:
df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STAT', 'ZKR': 'STAT_LABEL', 'ISO_ALPHA3': 'STAT_ISO'}), 
              how='left')
df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STATB', 'ZKR': 'STATB_LABEL', 'ISO_ALPHA3': 'STATB_ISO'}), 
              how='left')

In [149]:
df.sort_index(axis=1).head()

,APRO1,APRO2,APRO3,CHYV,DAT_REG,DAT_VYPR,DAT_ZAPS,FORMA_ST,IZOS,OBECB,...,ROD_KOD,STAT,STATB,STATB_ISO,STATB_LABEL,STAT_ISO,STAT_LABEL,TYP_ST,VYPR,ZAPS
0,,,,00 01 1 00000 0000000,3042017,03042017,31102017,P,999999840,,...,,840.0,840.0,USA,Spojené státy,USA,Spojené státy,B,10,2
1,,,,00 01 1 00000 0000000,9052017,09052017,31102017,P,999999368,,...,,368.0,368.0,IRQ,Irák,IRQ,Irák,B,10,2
2,,,,00 0 0 0000000 0000 0 0,22022017,15062017,,P,999999112,,...,000101,112.0,112.0,BLR,Bělorusko,BLR,Bělorusko,M,20,0
3,,,,00 0 0 0000000 0000 0 0,22022017,15062017,,P,999999112,,...,000101,112.0,112.0,BLR,Bělorusko,BLR,Bělorusko,M,20,0
4,,,,00 0 0 0000000 0000 0 0,13022017,20062017,,P,999999112,,...,000101,112.0,112.0,BLR,Bělorusko,BLR,Bělorusko,M,20,0


In [150]:
df['STATB_ISO'].value_counts()

CZE    217261
SVK     16512
RUS      4455
UKR      2737
KAZ      1774
        ...  
ATA         1
GAB         1
SXM         1
DOM         1
BTN         1
Name: STATB_ISO, Length: 171, dtype: int64

In [152]:
df.shape

(253584, 32)

In [268]:
li = []
for y in range(1999, 2023):
    pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
    li.append(pdf)
ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)

In [269]:
ss.head()

,IZO,ZAR_PLN,MISTO,ULICE,CP,PSC,VUSC,OBOR,OBOR_NAZ,PLN_NAZ,RED_IZO,ZAR_ZKR,POM
0,10189,Střední odborné učiliště strojírenské ...,Vsetín,Jasenická 1487,,75501,CZ0723,2343L506,Provozní technika,Provozní technika ...,NaN,NaN,NaN
1,10189,Střední odborné učiliště strojírenské ...,Vsetín,Jasenická 1487,,75501,CZ0723,2345L002,Mech.seř.-obr.str.a linky,Mechanik seřizovač - obráběcí stroje a linky ...,NaN,NaN,NaN
2,13757,Polygrafické SOU ...,Praha 10 - Petrovice,Bellova 352,,10900,CZ010A,3441L003,Polygraf,Polygraf ...,NaN,NaN,NaN
3,13757,Polygrafické SOU ...,Praha 10 - Petrovice,Bellova 352,,10900,CZ010A,3441L501,Polygrafický průmysl,Polygrafický průmysl ...,NaN,NaN,NaN
4,53155,Střední odborné učiliště ...,Bzenec,Vinařů 354,,69681,CZ0645,4144L002,Technik zahradnic.provozu,Technik zahradnického provozu ...,NaN,NaN,NaN


In [272]:
df['IZOS'] = pd.to_numeric(df['IZOS'], errors='coerce')

In [273]:
df['IZOS']

0         999999840.0
1         999999368.0
2         999999112.0
3         999999112.0
4         999999112.0
             ...     
253579    999999724.0
253580    999999124.0
253581    999999804.0
253582    999999398.0
253583    999999703.0
Name: IZOS, Length: 253584, dtype: float64

In [274]:
ss.shape

(34994, 13)

In [275]:
sss = ss[['IZO', 'ZAR_PLN', 'VUSC']].drop_duplicates()

In [276]:
sss.shape

(5696, 3)

### ODHL

In [201]:
# odkud se uchazeč hlásí - číselník MCPP - sloupec ODHL
mcpp_xml = 'http://stistko.uiv.cz/katalog/textdata/C213729MCPP.xml'
mcpp = pd.read_xml(mcpp_xml, encoding='cp1250', xpath='./veta')
mcpp

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,MCPP,1,Základní škola,Základní škola,15.4.1993,31.12.2999,NaN,NaN
1,MCPP,2,Střední škola,Střední škola,15.4.1993,31.12.2999,NaN,NaN
2,MCPP,3,Zaměstnání,Zaměstnání,15.4.1993,31.12.2999,NaN,NaN
3,MCPP,4,Domácnost,Domácnost,15.4.1993,31.12.2999,NaN,NaN
4,MCPP,6,Vyšší odborná škola,Vyšší odborná škola,30.6.1997,31.12.2999,NaN,NaN
5,MCPP,7,Vysoká škola,Vysoká škola,1.7.1997,31.12.2999,NaN,NaN
6,MCPP,8,Přichází přes DZS,Přichází přes Dům zahraničních služeb,21.1.1994,31.12.2999,NaN,NaN
7,MCPP,9,Jiné,Jiné,15.4.1993,31.12.2999,NaN,NaN
8,MCPP,C,Celoživ.vzdělávání VŠ,Program celoživotního vzdělávání na VŠ,1.1.2013,31.12.2999,NaN,NaN


In [210]:
foo = pd.merge(df, mcpp[['KOD', 'TXT']].rename(columns={'KOD': 'ODHL', 'TXT': 'ODHL_LABEL'}), how='left', indicator=True)

In [211]:
foo['_merge'].value_counts()

both          253331
left_only        253
right_only         0
Name: _merge, dtype: int64

In [205]:
mcpp['KOD'][5]

'7'

In [220]:
pd.to_numeric(df['RID'])

ValueError: Unable to parse string "7E900" at position 28

In [221]:
# fakulty - číselník regpro.dbf - sloupec RID

# with dbf.Table(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.dbf') as table:
#     dbf.export(table, f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv')
    
regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
regpro.head()

,RID,ZAR_PLN,VS_PLN,ZAR_NAZ,PSC,MISTO,ULICE,CP,SUZEMI,UZEMI,VS_ZKR,DAT_VZN,DAT_ZAH,DAT_AKT,DAT_KON,ID_FNAM,ID_SUCH
0,12520,Fakulta rybářství a ochrany vod ...,Jihočeská univerzita v Českých Budějovicích ...,Fakulta rybářství a ochrany vod,389 25,Vodňany,Zátiší 728/II,,3307,CZ0316,JU v Českých Budějovicích,2009-09-01,2009-09-01,None,None,BP0,1252
1,19510,Fakulta veřejných politik v Opavě ...,Slezská univerzita v Opavě ...,Fakulta veřejných politik v Opavě,746 01,Opava,Olbrichova 625/25,,3806,CZ0815,SU v Opavě,2008-10-01,2008-10-01,2010-06-21,None,IC0,1951
2,41320,Fakulta lesnická a dřevařská ...,Česká zemědělská univerzita v Praze ...,Fakulta lesnická a dřevařská,165 21,Praha 6 - Suchdol,Kamýcká 1176,,3106,CZ0116,ČZU v Praze,2007-07-01,None,2009-04-21,None,RF0,4132
3,15120,Fakulta zdravotnických věd ...,Univerzita Palackého v Olomouci ...,Fakulta zdravotnických věd,771 26,Olomouc 1,tř. Svobody 8,,3805,CZ0712,UP v Olomouci,None,None,None,None,ER0,1512
4,23310,Fakulta zdravotnických studií ...,Západočeská univerzita v Plzni ...,Fakulta zdravotnických studií,306 14,Plzeň,Univerzitní 8,,3405,CZ0323,ZČU v Plzni,2008-07-08,2008-07-08,None,None,LY0,2331


In [223]:
regpro['RID'][0]

'12520'

In [230]:
df.dtypes

RDAT                int64
RID                object
ROD_KOD            object
STATB             float64
OBECB              object
PSCB               object
STAT              float64
ODHL               object
IZOS               object
OBSS               object
RMAT              float64
TYP_ST             object
FORMA_ST           object
PROGRAM            object
OBOR1              object
OBOR2              object
OBOR3              object
OBOR4              object
OBOR5              object
APRO1              object
APRO2              object
APRO3              object
DAT_REG             int64
VYPR                int64
DAT_VYPR           object
ZAPS                int64
DAT_ZAPS           object
CHYV               object
STAT_LABEL         object
STAT_ISO           object
STATB_LABEL        object
STATB_ISO          object
OBSS_LABEL         object
ODHL_LABEL         object
TYP_ST_LABEL       object
FORMA_ST_LABEL     object
VYPR_LABEL         object
ZAPS_LABEL         object
RID_ZAR_PLN 

In [234]:
# aprobace - číselník AAAP - sloupce APRO[1-3]
aaap_xml = 'http://stistko.uiv.cz/katalog/textdata/C214234AAAP.xml'
aaap = pd.read_xml(aaap_xml, encoding='cp1250', xpath='./veta')
aaap.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AAAP,1,Předměty pro 1.stupeň ZŠ,Předměty a metodiky předmětů pro 1. stupeň zák...,15.4.1993,31.12.2999,1ST,NaN
1,AAAP,2,Český jazyk,Český jazyk,15.4.1993,31.12.2999,CJ,NaN
2,AAAP,3,Slovenský jazyk,Slovenský jazyk,15.4.1993,31.12.2999,SJ,NaN
3,AAAP,4,Maďarský jazyk,Maďarský jazyk,15.4.1993,31.12.2999,MJ,NaN
4,AAAP,5,Polský jazyk,Polský jazyk,15.4.1993,31.12.2999,PJ,NaN


In [235]:
aaap.dtypes

IDC       object
KOD        int64
ZKR       object
TXT       object
DATZP     object
DATKP     object
SPC       object
POZN     float64
dtype: object

In [239]:
program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
program.head()

,KOD,NAZEV,RID
0,B0111A190001,Pedagogika ...,14210
1,B0111A190002,Sociální pedagogika a volný čas ...,14410
2,B0111A190003,Andragogika ...,15210
3,B0111A190004,Andragogika a personální řízení ...,11210
4,B0111A190005,Sociální pedagogika a poradenství ...,14210


In [242]:
program['KOD'].drop_duplicates().shape

(2143,)

In [243]:
program['KOD'].str[:5].drop_duplicates().shape

(689,)

In [245]:
df['PROGRAM'].str.len().value_counts()

5    253584
Name: PROGRAM, dtype: int64

In [247]:
akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')

In [248]:
akko.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AKKO,1,PŘÍRODNÍ VĚDY A NAUKY,PŘÍRODNÍ VĚDY A NAUKY,1.9.1998,31.12.2999,NaN,None
1,AKKO,11,Matematické obory,Matematické obory,1.9.1998,31.12.2999,NaN,None
2,AKKO,1101R,Matematika,Matematika,1.9.1998,31.12.2999,NaN,None
3,AKKO,1101T,Matematika,Matematika,1.9.1998,31.12.2999,NaN,None
4,AKKO,1101V,Matematika,Matematika,1.9.1998,31.12.2999,NaN,None


In [250]:
df['OBOR1_5'] = df['OBOR1'].str[:5]

In [251]:
df['OBOR1_5']

0         6701R
1         8206R
2         5103T
3         5104T
4         5103T
          ...  
253579    5103T
253580    5103T
253581    6208R
253582    6107R
253583    6208R
Name: OBOR1_5, Length: 253584, dtype: object

In [253]:
df = pd.merge(df, akko[['KOD', 'TXT']].rename(columns={'KOD': 'OBOR1_5', 'TXT': 'OBOR1_5_LABEL'}), how='left')

In [254]:
df[(df['STAT_ISO'] == 'CZE') & (df['STATB_ISO'] == 'CZE') & (df['RMAT'] == 2017)].head(1000).to_csv('temp/uchazec/czmat_1000.csv')

### Střední školy IZOS, OBSS

In [182]:
df = loader()

/tmp/ipykernel_417/2491268812.py:6: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path17}.csv', encoding='cp1250')


In [166]:
df.shape

(253584, 32)

In [168]:
%%time

li = []

for y in range(1999, 2023):
    pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
    li.append(pdf)
    
ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)

CPU times: user 484 ms, sys: 0 ns, total: 484 ms
Wall time: 481 ms


In [169]:
def selector(frame):
    # just use the longest one - the shorter ones typically use abbreviations
    frame['len'] = frame['PLN_NAZ'].str.strip().str.len()
    return frame.sort_values('len', ascending=False)['PLN_NAZ'].iloc[0]

ss['OBOR'] = ss['OBOR'].str.strip()
obory = ss[['OBOR', 'PLN_NAZ']].drop_duplicates()
obory_uni = obory.groupby('OBOR')[['PLN_NAZ']].apply(selector).reset_index()
obory_uni

,OBOR,0
0,1601M001,Ochrana a tvorba životního prostředí ...
1,1601M002,Ochrana přírody a prostředí ...
2,1601M003,Životní prostředí ...
3,1601M004,Ekologie a ochrana krajiny ...
4,1601M005,Ochrana a obnova životního prostředí ...
...,...,...
1196,8251L05,Uměleckořemeslné zpracování skla ...
1197,8251L06,Uměleckořemeslná stavba hudebních nástrojů ...
1198,8251L501,Umělecké řemeslné práce ...
1199,8251L51,Umělecké řemeslné práce ...


In [170]:
df = pd.merge(df, obory_uni.rename(columns={'OBOR': 'OBSS', 0: 'OBSS_LABEL'}), how='left', indicator=True)
df.head()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV,STAT_LABEL,STAT_ISO,STATB_LABEL,STATB_ISO,OBSS_LABEL,_merge
0,20171031,78900,,840.0,,,840.0,2,999999840,,...,03042017,2,31102017,00 01 1 00000 0000000,Spojené státy,USA,Spojené státy,USA,NaN,left_only
1,20171031,78900,,368.0,,,368.0,2,999999368,,...,09052017,2,31102017,00 01 1 00000 0000000,Irák,IRQ,Irák,IRQ,NaN,left_only
2,20171031,15110,000101,112.0,,,112.0,2,999999112,,...,15062017,0,,00 0 0 0000000 0000 0 0,Bělorusko,BLR,Bělorusko,BLR,NaN,left_only
3,20171031,15110,000101,112.0,,,112.0,2,999999112,,...,15062017,0,,00 0 0 0000000 0000 0 0,Bělorusko,BLR,Bělorusko,BLR,NaN,left_only
4,20171031,14110,000101,112.0,,23104,112.0,9,999999112,,...,20062017,0,,00 0 0 0000000 0000 0 0,Bělorusko,BLR,Bělorusko,BLR,NaN,left_only


In [171]:
df.shape

(253584, 34)

In [120]:
df['RMAT'].value_counts()

2017    141267
2016     42102
2015     17958
2014     11992
2013      8585
         ...  
1965         1
1964         1
1961         1
1957         1
9999         1
Name: RMAT, Length: 62, dtype: int64

In [121]:
obory_uni

,OBOR,0
0,1601M001,Ochrana a tvorba životního prostředí ...
1,1601M002,Ochrana přírody a prostředí ...
2,1601M003,Životní prostředí ...
3,1601M004,Ekologie a ochrana krajiny ...
4,1601M005,Ochrana a obnova životního prostředí ...
...,...,...
1196,8251L05,Uměleckořemeslné zpracování skla ...
1197,8251L06,Uměleckořemeslná stavba hudebních nástrojů ...
1198,8251L501,Umělecké řemeslné práce ...
1199,8251L51,Umělecké řemeslné práce ...


In [190]:
df = pd.merge(df, obory_uni.rename(columns={'OBOR': 'OBSS', 0: 'OBSS_LABEL'}), how='left')
#df = pd.merge(df, obory_uni.rename(columns={'OBOR': 'OBSS', 0: 'OBSS_LABEL'}), how='left', indicator=True)
df.head()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV,STAT_LABEL,STAT_ISO,STATB_LABEL,STATB_ISO,OBSS_LABEL
0,20171031,78900,,840.0,,,840.0,2,999999840,,...,10,03042017,2,31102017,00 01 1 00000 0000000,Spojené státy,USA,Spojené státy,USA,NaN
1,20171031,78900,,368.0,,,368.0,2,999999368,,...,10,09052017,2,31102017,00 01 1 00000 0000000,Irák,IRQ,Irák,IRQ,NaN
2,20171031,15110,000101,112.0,,,112.0,2,999999112,,...,20,15062017,0,,00 0 0 0000000 0000 0 0,Bělorusko,BLR,Bělorusko,BLR,NaN
3,20171031,15110,000101,112.0,,,112.0,2,999999112,,...,20,15062017,0,,00 0 0 0000000 0000 0 0,Bělorusko,BLR,Bělorusko,BLR,NaN
4,20171031,14110,000101,112.0,,23104,112.0,9,999999112,,...,20,20062017,0,,00 0 0 0000000 0000 0 0,Bělorusko,BLR,Bělorusko,BLR,NaN


In [124]:
df.shape

(253584, 34)

In [125]:
df.head(10000).to_csv('temp/uchazec/df17_10000.csv')

In [156]:
df['RMAT'] = pd.to_numeric(df['RMAT'], errors='coerce')

In [134]:
df[(df['STAT_ISO'] == 'CZE') & (df['STATB_ISO'] == 'CZE') & (df['RMAT'] == 2017)].shape

(127003, 34)

In [192]:
df[(df['STAT_ISO'] == 'CZE') & (df['STATB_ISO'] == 'CZE') & (df['RMAT'] == 2017)].head(1000).to_csv('temp/uchazec/czmat_1000_.csv')

In [193]:
foo = df[(df['STAT_ISO'] == 'CZE') & (df['STATB_ISO'] == 'CZE') & (df['RMAT'] == 2017)]

In [194]:
foo.shape

(127003, 33)

In [198]:
pd.isna(foo['OBSS_LABEL']).sum()

644

In [195]:
np.sum(foo['OBSS_LABEL'] == '')

0

In [174]:
obory_uni[obory_uni['OBOR'].str.startswith('6842N')]['OBOR'].iloc[3]

'6842N02'

In [175]:
df[df['OBSS'].str.startswith('6842N')]['OBSS'].iloc[1]

'6842N02'

In [162]:
df[df['OBSS'].str.startswith('6842N')]['OBSS'].iloc[1] == obory_uni[obory_uni['OBOR'].str.startswith('6842N')]['OBOR'].iloc[3]

True

In [181]:
type(df['OBSS'].loc[1025])

str

In [178]:
df[df['OBSS'].str.startswith('6842N')]['OBSS']

942       6842N06
1025      6842N02
1026      6842N02
1260      6842N06
1431      6842N06
           ...   
97047     6842N04
136402    6842N04
155825    6842N04
222858    6842N04
246582    6842N04
Name: OBSS, Length: 75, dtype: object

In [100]:
df

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV,STAT_LABEL,STAT_ISO,STATB_LABEL,STATB_ISO,OBSS_LABEL
0,20171031,14110,0002241426,703.0,,82101,703.0,2,999999703,7941K801,...,10,20062017,2,31102017,00 0 0 0000000 0000000,Slovensko,SVK,Slovensko,SVK,Gymnázium - všeobecné (8leté) ...
1,20171031,14310,0002241426,703.0,,82101,703.0,2,999999703,7941K801,...,13,26072017,2,31102017,00 0 0 0000000 0000000,Slovensko,SVK,Slovensko,SVK,Gymnázium - všeobecné (8leté) ...
2,20171031,11230,00532111X9,643.0,,,643.0,2,000582875,7941K801,...,21,01062017,0,,0000 1 0000000 0000 0 0,Rusko,RUS,Rusko,RUS,Gymnázium - všeobecné (8leté) ...
3,20171031,25410,7659QQ6729,203.0,596868,59202,203.0,4,002943492,7941K801,...,21,18082017,0,,00000 0 0 0000000 0000 0 0,Česko,CZE,Česko,CZE,Gymnázium - všeobecné (8leté) ...
4,20171031,26210,7909QQ7218,203.0,536385,79001,203.0,3,102692173,7941K801,...,90,31102017,0,,00000 0 0 0000000 0000 0 0,Česko,CZE,Česko,CZE,Gymnázium - všeobecné (8leté) ...


In [183]:
df_ = df[df['OBSS'].str.startswith('6842N')].copy()

In [188]:
foo = pd.merge(df, obory_uni.rename(columns={'OBOR': 'OBSS', 0: 'OBSS_LABEL'}), how='left')

In [187]:
foo

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV,STAT_LABEL,STAT_ISO,STATB_LABEL,STATB_ISO,OBSS_LABEL
0,20171031,6K900,6410QQ3966,203.0,554642,35301,203.0,2,110030958,6842N06,...,10,06072017,1,09082017,000000000 0 0000000 0000000,Česko,CZE,Česko,CZE,Bezpečnost obyvatelstva ...
1,20171031,61900,6508QQ3965,203.0,554782,15200,203.0,2,110350201,6842N02,...,10,05102017,1,05102017,000000000 0 0000000 0000000,Česko,CZE,Česko,CZE,Krizové řízení ...
2,20171031,6K900,6508QQ3965,203.0,554782,15200,203.0,2,110350201,6842N02,...,90,12092017,0,,000000000 0 0000000 0000 0 0,Česko,CZE,Česko,CZE,Krizové řízení ...
3,20171031,6K900,6703QQ3967,203.0,554791,31200,203.0,2,110030958,6842N06,...,10,24072017,1,01092017,000000000 0 0000000 0000000,Česko,CZE,Česko,CZE,Bezpečnost obyvatelstva ...
4,20171031,6K900,6807QQ3966,203.0,559067,33144,203.0,2,110030958,6842N06,...,10,05072017,1,11082017,000000000 0 0000000 0000000,Česko,CZE,Česko,CZE,Bezpečnost obyvatelstva ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,20171031,13440,9703QQ9073,203.0,554804,40011,203.0,2,110018095,6842N04,...,90,23062017,0,,0000000 10 00000000 0000 0 0,Česko,CZE,Česko,CZE,NaN
71,20171031,18900,9755QQ1105,203.0,571041,50346,203.0,2,151014655,6842N04,...,0,,0,,0000000 10 0000000 0000 0,Česko,CZE,Česko,CZE,NaN
72,20171031,17110,9759QQ9380,203.0,598810,73955,203.0,2,200000055,6842N04,...,20,20062017,0,,0000000 10 0000000 0000 0 0,Česko,CZE,Česko,CZE,NaN
73,20171031,13430,9854QQ9850,203.0,567337,43521,203.0,2,108006891,6842N04,...,90,22062017,0,,0000000 10 0000000 0000 0 0,Česko,CZE,Česko,CZE,NaN


In [189]:
foo[foo['OBSS'].str.startswith('6842N')].copy()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV,STAT_LABEL,STAT_ISO,STATB_LABEL,STATB_ISO,OBSS_LABEL
942,20171031,6K900,6410QQ3966,203.0,554642,35301,203.0,2,110030958,6842N06,...,10,06072017,1,09082017,000000000 0 0000000 0000000,Česko,CZE,Česko,CZE,Bezpečnost obyvatelstva ...
1025,20171031,61900,6508QQ3965,203.0,554782,15200,203.0,2,110350201,6842N02,...,10,05102017,1,05102017,000000000 0 0000000 0000000,Česko,CZE,Česko,CZE,Krizové řízení ...
1026,20171031,6K900,6508QQ3965,203.0,554782,15200,203.0,2,110350201,6842N02,...,90,12092017,0,,000000000 0 0000000 0000 0 0,Česko,CZE,Česko,CZE,Krizové řízení ...
1260,20171031,6K900,6703QQ3967,203.0,554791,31200,203.0,2,110030958,6842N06,...,10,24072017,1,01092017,000000000 0 0000000 0000000,Česko,CZE,Česko,CZE,Bezpečnost obyvatelstva ...
1431,20171031,6K900,6807QQ3966,203.0,559067,33144,203.0,2,110030958,6842N06,...,10,05072017,1,11082017,000000000 0 0000000 0000000,Česko,CZE,Česko,CZE,Bezpečnost obyvatelstva ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97047,20171031,13440,9703QQ9073,203.0,554804,40011,203.0,2,110018095,6842N04,...,90,23062017,0,,0000000 10 00000000 0000 0 0,Česko,CZE,Česko,CZE,NaN
136402,20171031,18900,9755QQ1105,203.0,571041,50346,203.0,2,151014655,6842N04,...,0,,0,,0000000 10 0000000 0000 0,Česko,CZE,Česko,CZE,NaN
155825,20171031,17110,9759QQ9380,203.0,598810,73955,203.0,2,200000055,6842N04,...,20,20062017,0,,0000000 10 0000000 0000 0 0,Česko,CZE,Česko,CZE,NaN
222858,20171031,13430,9854QQ9850,203.0,567337,43521,203.0,2,108006891,6842N04,...,90,22062017,0,,0000000 10 0000000 0000 0 0,Česko,CZE,Česko,CZE,NaN


In [104]:
df.shape

(26949, 33)

In [103]:
df['RMAT'].value_counts()

2012    4411
2017    3390
2011    2960
2010    2005
2013    1543
2009    1505
2014    1033
2008     939
2007     883
2016     877
2015     848
2006     728
2005     709
2004     645
2003     559
2002     495
2001     465
1998     390
1999     382
1997     267
2000     242
1996     223
1995     202
1994     188
1993     174
1992     157
1991     154
1990     123
1989      91
1988      66
1986      61
1987      61
1985      41
1983      36
1984      27
1982      20
1981      13
1979       8
1978       6
1980       5
2018       4
1976       3
1977       3
1970       2
           2
1961       1
1971       1
1974       1
Name: RMAT, dtype: int64

### Střední školy IZOBRMAT

In [57]:
# střední školy - číselník izobrmat.dbf - sloupce IZOS, OBSS

# with dbf.Table(f'{data_root}/uchazec/uch3-03-6/cisel/izobrmat.dbf') as table:
#     dbf.export(table, f'{data_root}/uchazec/uch3-03-6/cisel/izobrmat.csv')
    
izobrmat = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/izobrmat.csv', encoding='cp1250')
izobrmat.head()

,IZO,OBOR,RMAT
0,10189,2343L506,1999
1,10189,2343L506,2000
2,10189,2343L506,2002
3,10189,2343L506,2003
4,10189,2345L001,2001


In [ ]:
df['']

In [60]:
df['OBSS'].value_counts().reset_index().to_csv('temp/uchazec/obss_counts.csv')

In [61]:
foo = df['OBSS'].value_counts().reset_index()

In [63]:
foo[foo['index'].str.contains('K')].to_csv('temp/uchazec/obss_K_counts.csv')

### Střední školy IZOS, OBSS

In [49]:
%%time

li = []

for y in range(1999, 2023):
    pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
    li.append(pdf)
    
ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)

CPU times: user 656 ms, sys: 125 ms, total: 781 ms
Wall time: 1.09 s


In [50]:
ss

,IZO,ZAR_PLN,MISTO,ULICE,CP,PSC,VUSC,OBOR,OBOR_NAZ,PLN_NAZ,RED_IZO,ZAR_ZKR,POM
0,10189,Střední odborné učiliště strojírenské ...,Vsetín,Jasenická 1487,,75501,CZ0723,2343L506,Provozní technika,Provozní technika ...,NaN,NaN,NaN
1,10189,Střední odborné učiliště strojírenské ...,Vsetín,Jasenická 1487,,75501,CZ0723,2345L002,Mech.seř.-obr.str.a linky,Mechanik seřizovač - obráběcí stroje a linky ...,NaN,NaN,NaN
2,13757,Polygrafické SOU ...,Praha 10 - Petrovice,Bellova 352,,10900,CZ010A,3441L003,Polygraf,Polygraf ...,NaN,NaN,NaN
3,13757,Polygrafické SOU ...,Praha 10 - Petrovice,Bellova 352,,10900,CZ010A,3441L501,Polygrafický průmysl,Polygrafický průmysl ...,NaN,NaN,NaN
4,53155,Střední odborné učiliště ...,Bzenec,Vinařů 354,,69681,CZ0645,4144L002,Technik zahradnic.provozu,Technik zahradnického provozu ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34989,181078988,Soukromá střední škola zemědělská s.r.o. ...,Pozďatín,č.p. 83,,67503,CZ0634,4143M02,Chovatelství,Chovatelství ...,691009465.0,Soukromá střední škola zemědělská s.r.o.,NaN
34990,181088231,"Křesťanská střední škola, základní škola a mat...",Praha 4 - Michle,Baarova 360/24,,14000,CZ0104,7842M06,Kombinované lyceum,Kombinované lyceum ...,651040001.0,"Křesťanská SŠ, ZŠ a MŠ Elijáš",NaN
34991,181096790,Waldorfská škola České Budějovice - mateřská š...,České Budějovice,M. Chlajna 1347/23,,37005,CZ0311,7842M06,Kombinované lyceum,Kombinované lyceum ...,651040621.0,"Waldorfská škola ČB - MŠ, ZŠ a SŠ o.p.s.",NaN
34992,181113601,Základní škola a Střední škola Donum Felix s.r...,Kladno,Petra Bezruče 3087,,27201,CZ0203,7842M03,Pedagogické lyceum,Pedagogické lyceum ...,691008167.0,ZŠ a SŠ Donum Felix s.r.o.,NaN


In [64]:
obory = ss[['OBOR', 'PLN_NAZ']].drop_duplicates()
obory

,OBOR,PLN_NAZ
0,2343L506,Provozní technika ...
1,2345L002,Mechanik seřizovač - obráběcí stroje a linky ...
2,3441L003,Polygraf ...
3,3441L501,Polygrafický průmysl ...
4,4144L002,Technik zahradnického provozu ...
...,...,...
34803,4331N01,Fyzioterapie zvířat ...
34805,2641N04,"Výroba,přenos el.energie ..."
34812,8242N09,"Konz.,rest.předm.ze dřeva ..."
34814,9111N01,"Ochr.vojsk,obyvatelstva ..."


In [65]:
obory[['OBOR']].drop_duplicates()

,OBOR
0,2343L506
1,2345L002
2,3441L003
3,3441L501
4,4144L002
...,...
34756,2647N24
34762,7943K61
34777,6341N31
34803,4331N01


In [66]:
obory.to_csv('temp/uchazec/obory.csv')

In [87]:
def selector(frame):
    frame['len'] = frame['PLN_NAZ'].str.strip().str.len()
    return frame.sort_values('len', ascending=False)['PLN_NAZ'].iloc[0]

In [89]:
obory_uni = obory.groupby('OBOR')[['PLN_NAZ']].apply(selector).reset_index()

In [90]:
obory_uni

,OBOR,0
0,1601M001,Ochrana a tvorba životního prostředí ...
1,1601M002,Ochrana přírody a prostředí ...
2,1601M003,Životní prostředí ...
3,1601M004,Ekologie a ochrana krajiny ...
4,1601M005,Ochrana a obnova životního prostředí ...
...,...,...
1196,8251L05,Uměleckořemeslné zpracování skla ...
1197,8251L06,Uměleckořemeslná stavba hudebních nástrojů ...
1198,8251L501,Umělecké řemeslné práce ...
1199,8251L51,Umělecké řemeslné práce ...


In [95]:
df['OBSS'] = df['OBSS'].str.strip()

In [97]:
df = pd.merge(df, obory_uni.rename(columns={'OBOR': 'OBSS', 0: 'OBSS_LABEL'}))

In [99]:
df.head()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV,STAT_LABEL,STAT_ISO,STATB_LABEL,STATB_ISO,OBSS_LABEL
0,20171031,14110,0002241426,703.0,,82101,703.0,2,999999703,7941K801,...,10,20062017,2,31102017,00 0 0 0000000 0000000,Slovensko,SVK,Slovensko,SVK,Gymnázium - všeobecné (8leté) ...
1,20171031,14310,0002241426,703.0,,82101,703.0,2,999999703,7941K801,...,13,26072017,2,31102017,00 0 0 0000000 0000000,Slovensko,SVK,Slovensko,SVK,Gymnázium - všeobecné (8leté) ...
2,20171031,11230,00532111X9,643.0,,,643.0,2,000582875,7941K801,...,21,01062017,0,,0000 1 0000000 0000 0 0,Rusko,RUS,Rusko,RUS,Gymnázium - všeobecné (8leté) ...
3,20171031,25410,7659QQ6729,203.0,596868,59202,203.0,4,002943492,7941K801,...,21,18082017,0,,00000 0 0 0000000 0000 0 0,Česko,CZE,Česko,CZE,Gymnázium - všeobecné (8leté) ...
4,20171031,26210,7909QQ7218,203.0,536385,79001,203.0,3,102692173,7941K801,...,90,31102017,0,,00000 0 0 0000000 0000 0 0,Česko,CZE,Česko,CZE,Gymnázium - všeobecné (8leté) ...


In [74]:
obory['PLN_NAZ'].str.strip().str.len()

0        17
1        44
2         8
3        20
4        29
         ..
34803    19
34805    24
34812    25
34814    23
34872    21
Name: PLN_NAZ, Length: 1336, dtype: int64

In [68]:
obory['PLN_NAZ'].str.len()

0        80
1        80
2        80
3        80
4        80
         ..
34803    80
34805    80
34812    80
34814    80
34872    80
Name: PLN_NAZ, Length: 1336, dtype: int64

In [51]:
ss.to_csv('temp/uchazec/ss.csv')

In [52]:
ss[ss['OBOR'].str.contains('K')].to_csv('temp/uchazec/ss_K.csv')

In [53]:
ss_K = ss[ss['OBOR'].str.contains('K')].copy()

In [56]:
ss_K[['OBOR', 'PLN_NAZ']].drop_duplicates().sort_values('OBOR').to_csv('temp/uchazec/ss_K_uni.csv')

In [47]:
cze = df[df['STATB_ISO'] == 'CZE']

In [48]:
cze

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,DAT_REG,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV,STAT_LABEL,STAT_ISO,STATB_LABEL,STATB_ISO
46,20171031,12310,000118,203.0,0,0,40.0,2,999999040,,...,26092017,13,29092017,1,29092017,100 0 0 1 0000000 0000000,Rakousko,AUT,Česko,CZE
56,20171031,63900,000126,203.0,534676,27601,804.0,4,999999804,,...,1062017,10,29082017,2,31102017,000 0 0 0 0000000 0000000,Ukrajina,UKR,Česko,CZE
68,20171031,26230,000208,203.0,540765,25210,804.0,2,999999804,,...,26022017,32,31102017,0,,000 0 0 0 0000000 0000 0 0,Ukrajina,UKR,Česko,CZE
124,20171031,11150,0003151533,203.0,582786,60200,804.0,2,999999804,,...,20022017,21,22062017,0,,000 0 0 0 0000000 0000 0 0,Ukrajina,UKR,Česko,CZE
125,20171031,11150,0003151533,203.0,582786,60200,804.0,2,999999804,,...,20022017,21,22062017,0,,000 0 0 0 0000000 0000 0 0,Ukrajina,UKR,Česko,CZE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253537,20171031,21230,9962QQ9089,203.0,554782,15500,804.0,2,999999804,,...,29032017,21,02102017,0,,000 0 0 0 000000 0000 0 0,Ukrajina,UKR,Česko,CZE
253541,20171031,14310,996208,203.0,563889,46312,100.0,2,999999804,,...,15022017,20,19052017,0,,000 0 0 0 0000000 0000 0 0,Bulharsko,BGR,Česko,CZE
253555,20171031,79900,996219123,203.0,,06000,203.0,2,,,...,30012017,20,30012017,0,,0110010 11 000000 0000 0 0,Česko,CZE,Česko,CZE
253575,20171031,26110,996225,203.0,554782,13000,762.0,8,999999762,,...,20022017,10,31102017,2,31102017,000 0 0 0 000000 0000000,Tádžikistán,TJK,Česko,CZE


In [30]:
df.sort_values('STAT').tail()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,DAT_REG,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV,STAT_LABEL,STAT_ISO,STATB_LABEL,STATB_ISO
19796,20171031,76900,8955QQ9790,NaN,529303,25601,NaN,9,,,...,11092017,10,11092017,1,20092017,10 11 1 00000 0000000,NaN,NaN,NaN,NaN
33936,20171031,6K900,9308QQ9758,NaN,,33301,NaN,2,110100662,2341M001,...,14032017,90,14032017,0,,0 100100 0000000 0000 0 0,NaN,NaN,NaN,NaN
34910,20171031,6F900,9311QQ9712,NaN,,L-805,NaN,2,999999703,,...,19092017,90,19092017,0,,10 1 0 0000000 0000 0 0,NaN,NaN,NaN,NaN
74605,20171031,6K900,9610QQ8040,NaN,,37842,NaN,2,,6842M01,...,13032017,10,13032017,1,06092017,1011 1 0000000 0000000,NaN,NaN,NaN,NaN
134297,20171031,7G900,9754QQ9286,NaN,,62800,NaN,2,044991517,6542M01,...,28022017,90,28022017,0,,0 100100 0000000 0000 0 0,NaN,NaN,NaN,NaN


In [29]:
df.shape

(253584, 32)

In [12]:
aast.dtypes

IDC           object
KOD            int64
ZKR           object
TXT           object
DATZP         object
DATKP         object
SPC           object
POZN          object
ISO_ALPHA3    object
dtype: object

In [13]:
df.dtypes

RDAT           int64
RID           object
ROD_KOD       object
STATB         object
OBECB         object
PSCB          object
STAT          object
ODHL          object
IZOS          object
OBSS          object
RMAT          object
TYP_ST        object
FORMA_ST      object
PROGRAM       object
OBOR1         object
OBOR2         object
OBOR3         object
OBOR4         object
OBOR5         object
APRO1         object
APRO2         object
APRO3         object
DAT_REG        int64
VYPR           int64
DAT_VYPR      object
ZAPS           int64
DAT_ZAPS      object
CHYV          object
STAT_LABEL    object
STAT_ISO      object
dtype: object

0         840
1         368
2         112
3         112
4         112
         ... 
253579    724
253580    300
253581    804
253582    398
253583    703
Name: STAT, Length: 253584, dtype: object